In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import numpy as np
import pandas as pd
import gc

In [23]:
path = "../Data/"
print("Read train file to obtain some statistic about the features and target")
train = pd.read_csv(path + 'renamed_train_2016.csv', parse_dates=['date'])
train = train.drop("id", axis=1)

prop = pd.read_csv(path + 'renamed_properties_2016.csv', usecols=["id_parcel", 'latitude', 'longitude', 'region_city'])
train = pd.merge(train, prop, on="id_parcel", how="left")
del prop
gc.collect()

Read train file to obtain some statistic about the features and target


38

In [4]:
train.head()

,id_parcel,logerror,date,latitude,longitude,region_city
0,11016594,0.0276,2016-01-01,34280990.0,-118488536.0,12447.0
1,14366692,-0.1684,2016-01-01,33668120.0,-117677556.0,32380.0
2,12098116,-0.0040,2016-01-01,34136312.0,-118175032.0,47019.0
3,12643413,0.0218,2016-01-02,33755800.0,-118309000.0,12447.0
4,14432541,-0.0050,2016-01-02,33485643.0,-117700234.0,17686.0


In [30]:
train.loc[:,'latitude'] = train.loc[:,'latitude']/1e6
train.loc[:,'longitude'] = train.loc[:,'longitude']/1e6

In [6]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [7]:
kmeans = MiniBatchKMeans(n_clusters=170, batch_size=1000).fit(train[['latitude','longitude']])

In [8]:
# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import folium
#from fbprophet import Prophet
plt.style.use('fivethirtyeight')
import pickle
import gc
import warnings
warnings.filterwarnings("ignore")

In [9]:
map_2 = folium.Map(location=[34.088537, -118.249923], zoom_start=9)

for label in kmeans.cluster_centers_:
    folium.Marker(location=[label][0]).add_to(map_2)

map_2

In [10]:
index = pd.isnull(train["region_city"])
train = train[~index]
print train.shape

(88472, 6)


In [11]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
lbl.fit(list(train["region_city"].values))
train["region_city"] = lbl.transform(list(train["region_city"].values))

In [12]:
lat = pd.DataFrame(train.groupby("region_city")["latitude"].mean())
lon = pd.DataFrame(train.groupby("region_city")["longitude"].mean())

In [13]:
lat["longitude"] = lon["longitude"].values

In [15]:
map_3 = folium.Map(location=[34.088537, -118.249923], zoom_start=9)

for ii, row in lat.iterrows():
    folium.Marker(location=[row["latitude"], row["longitude"]]).add_to(map_3) 
    
map_3

In [69]:
df = train[train["region_city"].isnull()]

In [46]:
map_4 = folium.Map(location=[34.088537, -118.249923], zoom_start=9)

for ii, row in df.iterrows():
    folium.CircleMarker(location=[row["latitude"], row["longitude"]], radius=.1, 
                        color='#0000ff').add_to(map_4) 
    
map_4

In [47]:
index = pd.isnull(train["region_city"])
df = train[~index]
print df.shape

(90275, 6)


In [49]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
lbl.fit(list(df["region_city"].values))
df["region_city"] = lbl.transform(list(df["region_city"].values))

In [50]:
lat = pd.DataFrame(train.groupby("region_city")["latitude"].mean())
lon = pd.DataFrame(train.groupby("region_city")["longitude"].mean())

lat["longitude"] = lon["longitude"].values

In [73]:
map_3 = folium.Map(location=[34.088537, -118.249923], zoom_start=9)

for ii, row in lat.iterrows():
    folium.CircleMarker(location=[row["latitude"], row["longitude"]], radius=4, 
                        color='#00ff00', fill_color='#00ff00').add_to(map_3) 
    folium.CircleMarker(location=[row["latitude"], row["longitude"]], radius=2, 
                        color='#00ff00', fill_color='#00ff00').add_to(map_3)
    
for ii, row in df.iterrows():
    folium.CircleMarker(location=[row["latitude"], row["longitude"]], radius=.1, 
                        color='#0000ff').add_to(map_3) 
map_3